You are in charge of developing a framework to detect fraud for a life insurance company. The life
insurance company services many the entire nation, but you will only be working with customer data
from their west coast branches. The company has never done any fraud modeling in the past, so you will
be starting from scratch.
  
One of the problems when building a fraud detection system from scratch is not knowing any fraudulent
claims to try and build a model from. In this situation you must use unsupervised learning or anomaly
detection techniques followed by domain expertise to try and evaluate where fraud might exist. Your
task is to identify groups of observations (customers in this case) you are suspicious of fraudulent acts.
You will submit these groups of observations. A “domain expert” will look through your groups and
determine if those groups exhibit fraudulent behavior. The group that can identify the most actual
fraudulent cases (I know the true flags of each observation) will have their group’s anomalies used for
modeling in the subsequent stages.  
  
**Identify no more than 2000 observations that you believe are potentially fraudulent.**

In [90]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

## Data Prep

In [91]:
df = pd.read_csv('aggregated_policy_nolabel_2025.csv')
print(df.shape)
df.head()

(44488, 120)


,GivenName,MiddleInitial,Surname,StreetAddress,City,State,ZipCode,Country,TelephoneNumber,MothersMaiden,...,Diff_Cust_Inc_CZIPInc,Avg_CH_CovLimit,IN_LessThan_60,Ratio_AdjZIP_PopZIP,Num_Adj_CustID,Zip_per_CustID,Avg_CL,PerSize_Max_CH,SD_CovLim_Last_6mon,Time_MaritalCH_CL
0,James,M,Baker,1782 Freedom Lane,Atwater,CA,95301,US,209-356-7271,McDonald,...,12897,0.0,0,0.001105926,1,1,50000,0.0,0.0,NaN
1,Melanie,C,Johnson,3230 Gateway Avenue,Lancaster,CA,93534,US,661-945-1609,Johnson,...,12108,0.0,0,0.001855571,1,1,100000,0.0,0.0,NaN
2,Anne,T,Bailey,1011 Maple Street,Anaheim,CA,92801,US,714-330-5620,Lehoux,...,-12715,0.0,0,0.000982793,1,1,100000,0.0,0.0,0.0
3,Geoffrey,E,Veiga,2370 Roosevelt Street,San Francisco,CA,94103,US,415-436-5690,Johnson,...,25636,0.0,0,0.001729849,1,1,50000,0.0,0.0,NaN
4,Sylvia,W,Hines,4404 Lindale Avenue,Oakland,CA,94607,US,510-551-8237,Woodward,...,72144,0.0,0,0.002562255,1,1,150000,0.0,0.0,NaN


Alot of this looks like junk, lets drop the junk that would have nothing to do with fraud.

In [92]:
junk = ['GivenName', 'MiddleInitial', 'Surname', 'StreetAddress', 'City', 'State',
    'ZipCode', 'Country', 'TelephoneNumber', 'MothersMaiden', 'Birthday',
    'BloodType', 'Pounds', 'FeetInches', 'Date_Initial', 'Cov_ID']
df = df.drop(columns=junk)
df

,Cust_ID,Type,Reward_Reason,Reward_Amount,Tobacco_IN,Caffeine_IN,Alcohol_IN,Number_Changes,Cov_Limit_Claim,Income_Claim,...,Diff_Cust_Inc_CZIPInc,Avg_CH_CovLimit,IN_LessThan_60,Ratio_AdjZIP_PopZIP,Num_Adj_CustID,Zip_per_CustID,Avg_CL,PerSize_Max_CH,SD_CovLim_Last_6mon,Time_MaritalCH_CL
0,PSX000100016,T,265,50000,N,Y,Y,0,50000,61000,...,12897,0.0,0,0.001105926,1,1,50000,0.0,0.0,NaN
1,PSX000101324,V,265,100000,N,Y,N,0,100000,48000,...,12108,0.0,0,0.001855571,1,1,100000,0.0,0.0,NaN
2,PSX000102929,V,450,100000,Y,Y,N,0,100000,35000,...,-12715,0.0,0,0.000982793,1,1,100000,0.0,0.0,0.0
3,PSX000104488,T,543,0,N,Y,Y,0,50000,69000,...,25636,0.0,0,0.001729849,1,1,50000,0.0,0.0,NaN
4,PSX000106040,T,200,150000,Y,Y,Y,0,150000,105000,...,72144,0.0,0,0.002562255,1,1,150000,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44483,PSX000999946,T,190,150000,N,Y,N,0,150000,29000,...,965,0.0,0,0.001377839,1,1,150000,0.0,0.0,NaN
44484,PSX000999949,V,527,0,N,N,Y,0,100000,39000,...,17970,0.0,0,0.003071356,1,1,100000,0.0,0.0,NaN
44485,PSX000999966,V,548,0,N,Y,Y,0,50000,83000,...,34486,0.0,0,0.002382459,1,1,50000,0.0,0.0,NaN
44486,PSX000999976,V,406,150000,N,Y,Y,0,150000,63000,...,22164,0.0,0,0.000202821,1,1,150000,0.0,0.0,NaN


Lets now investigate missingness

In [93]:
# columns with missing data
missing_values = df.isnull().sum()
print("Columns with missing data:\n", missing_values[missing_values > 0])

Columns with missing data:
 Adj_ZIP                    112
Distance_Claim_Adj        6429
Time_LastChange_Claim    23311
Marriage                 37858
Pop_CZIP                  1805
MedInc_CZIP               2214
Time_MaritalCH_CL        43729
dtype: int64


'Time_LastChange_Claim', 'Marriage', 'Time_MaritalCH_CL' are all missing more often than not so lets drop them

In [94]:
missing_more_50_per = ['Time_LastChange_Claim', 'Marriage', 'Time_MaritalCH_CL']
df = df.drop(columns=missing_more_50_per)

that leaves us with the following missing columns. I notice that they all are relaetd to zip codes and would love to go through this data and impute the correct zip codes where they are missing; however, give the shiort turn around of this assignment I will just be dropping these columns.

In [95]:
remaning_cols = ['Adj_ZIP', 'Distance_Claim_Adj', 'Pop_CZIP', 'MedInc_CZIP']
df = df.drop(columns=remaning_cols)

Lets now do some other small tweaks

In [96]:
# set the customer idf as index
df.set_index('Cust_ID', inplace=True)

# replace all 'Y' values with 1 and all 'N' values with 0
df.replace({'Y': 1, 'N': 0}, inplace=True) 

# For compute times lets drop columns with more than 50 unique categories - ik this isnt best practice but this assignment is rushed
df.drop(
    columns=[col for col in df.select_dtypes(include=['object']).columns if df[col].nunique() > 50],
    inplace=True
)

df.head()

C:\Users\Sterling Hayden\AppData\Local\Temp\ipykernel_11976\995320745.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'Y': 1, 'N': 0}, inplace=True)


,Type,Reward_Reason,Reward_Amount,Tobacco_IN,Caffeine_IN,Alcohol_IN,Number_Changes,Cov_Limit_Claim,Income_Claim,Adj_Tech_Confidence,...,R_Greater_CL,Ratio_CH_TimeCompany,CL_per_Adj,Avg_CH_CovLimit,IN_LessThan_60,Num_Adj_CustID,Zip_per_CustID,Avg_CL,PerSize_Max_CH,SD_CovLim_Last_6mon
Cust_ID,,,,,,,,,,,,,,,,,,,,,
PSX000100016,T,265,50000,0,1,1,0,50000,61000,0.072289,...,0,0.0,70,0.0,0,1,1,50000,0.0,0.0
PSX000101324,V,265,100000,0,1,0,0,100000,48000,0.058333,...,0,0.0,101,0.0,0,1,1,100000,0.0,0.0
PSX000102929,V,450,100000,1,1,0,0,100000,35000,0.043478,...,0,0.0,83,0.0,0,1,1,100000,0.0,0.0
PSX000104488,T,543,0,0,1,1,0,50000,69000,0.148148,...,0,0.0,70,0.0,0,1,1,50000,0.0,0.0
PSX000106040,T,200,150000,1,1,1,0,150000,105000,0.031579,...,0,0.0,83,0.0,0,1,1,150000,0.0,0.0


Finally lets prep the data for our model

In [97]:
# Convert categorical columns
df = pd.get_dummies(df, drop_first=False)

# Scale features - from what ive read I have heard that its best practice to scale when doing anomaly detection 
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

## Isolation Forest Model

In [98]:
clf = IsolationForest(contamination=5000/df.shape[0],random_state=88)
anomalies = clf.fit_predict(df_scaled)

# Add anomaly labels to DataFrame (-1 = anomaly, 1 = normal)
df['anomaly'] = anomalies
df.head()

,Reward_Reason,Reward_Amount,Tobacco_IN,Caffeine_IN,Alcohol_IN,Number_Changes,Cov_Limit_Claim,Income_Claim,Adj_Tech_Confidence,Tech_Adj_Confidence,...,SD_CovLim_Last_6mon,Type_T,Type_V,Type_W,Med_Conditions_0,Med_Conditions_1,Med_Conditions_2,Med_Conditions_3,Med_Conditions_3+,anomaly
Cust_ID,,,,,,,,,,,,,,,,,,,,,
PSX000100016,265,50000,0,1,1,0,50000,61000,0.072289,0.062500,...,0.0,True,False,False,False,False,True,False,False,1
PSX000101324,265,100000,0,1,0,0,100000,48000,0.058333,0.068627,...,0.0,False,True,False,True,False,False,False,False,1
PSX000102929,450,100000,1,1,0,0,100000,35000,0.043478,0.038462,...,0.0,False,True,False,False,True,False,False,False,1
PSX000104488,543,0,0,1,1,0,50000,69000,0.148148,0.125000,...,0.0,True,False,False,False,True,False,False,False,1
PSX000106040,200,150000,1,1,1,0,150000,105000,0.031579,0.015385,...,0.0,True,False,False,False,True,False,False,False,1


## Compare against classmates results

In [99]:
# grab Cust_ID of identified anomalies
fraudsters = df[df['anomaly']==-1].index
fraudsters # there are duplicate cust ids in this

Index(['PSX00011227', 'PSX000113163', 'PSX00011382', 'PSX0001176',
       'PSX000118762', 'PSX000121923', 'PSX000123474', 'PSX000123512',
       'PSX000124213', 'PSX000124489',
       ...
       'PSX00099855', 'PSX000999020', 'PSX000999193', 'PSX000999267',
       'PSX000999372', 'PSX000999383', 'PSX000999504', 'PSX00099955',
       'PSX000999588', 'PSX000999804'],
      dtype='object', name='Cust_ID', length=5000)

lets see which customer id's me and my teamates share

In [100]:
# load in teamates set of ids
ishi =  pd.read_csv('ishi_2.csv')
ishi.set_index('Cust_ID', inplace=True)
ishi = ishi.index
jackson = pd.read_excel('jackson.xlsx')
jackson.set_index('Cust_ID', inplace=True)
jackson = jackson.index
brian = pd.read_csv('brian_2.csv')
brian.set_index('Cust_ID', inplace=True)
brian = brian.index
van = pd.read_csv('van_2.csv')
van.set_index('Cust_ID', inplace=True)
van = van.index


# see the intersection in which we both share the same ids
# common_ids = fraudsters.intersection(jackson).intersection(brian).intersection(ishi).intersection(van)
common_ids = fraudsters.intersection(ishi).intersection(van).intersection(brian).intersection(jackson)
common_ids

Index(['PSX000177427', 'PSX000245305', 'PSX000245549', 'PSX000245755',
       'PSX000246035', 'PSX000246768', 'PSX000247764', 'PSX000250441',
       'PSX00025304', 'PSX000253191',
       ...
       'PSX000985458', 'PSX000986786', 'PSX000988616', 'PSX00099040',
       'PSX000990504', 'PSX000990528', 'PSX000990568', 'PSX000995560',
       'PSX000996356', 'PSX000999588'],
      dtype='object', name='Cust_ID', length=339)

In [101]:
# convert to df and save to csv to send off to the 'domain expert'
df_common_ids = pd.DataFrame(common_ids, columns=['Cust_ID'])
df_common_ids.to_csv('blue-14-assignment-2.csv', index=False)